<a href="https://colab.research.google.com/github/dmswl0707/CoronaCoprosperityCard_DataScience/blob/main/Card_Payment_Ratio_by_Age.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import json

from google.colab import drive
drive.mount('/content/drive') #구글 드라이브에 접근
filename='/content/drive/My Drive/Colab Notebooks/data/seoul2020.csv' #파일 경로설정

data=pd.read_csv(filename1, encoding='CP949') #csv 파일 불러오기
print(data.head(10)) 
print(data.info()) #파일정보
print(data.index)
print(data.columns)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   기준_년_코드  기준_분기_코드 상권_구분_코드  ... 연령대_50_매출_건수  연령대_60_이상_매출_건수 점포수
0     2020         2        U  ...            5                5   8
1     2020         2        U  ...         1062             1003   8
2     2020         2        U  ...         1722             1862   5
3     2020         2        U  ...         2101              781   6
4     2020         2        U  ...         4241             1235   6
5     2020         2        U  ...          387              149   8
6     2020         2        U  ...         3479             2861  14
7     2020         2        U  ...        15452             5373  54
8     2020         2        U  ...        14868             7207  12
9     2020         2        U  ...        47000            28634   8

[10 rows x 80 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66511 entries, 0 to 66510
Data columns

In [112]:
data_null=np.sum(pd.isnull(data)) #결측치 확인
#data_null.tolist()

In [113]:
###필요한 index 추출
 
data_sample1=data[['연령대_10_매출_건수','연령대_20_매출_건수','연령대_30_매출_건수','연령대_40_매출_건수','연령대_50_매출_건수','연령대_60_이상_매출_건수','연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액',
       '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액']]

print(data_sample1.head(100))



    연령대_10_매출_건수  연령대_20_매출_건수  ...  연령대_50_매출_금액  연령대_60_이상_매출_금액
0              0            10  ...        155459           170033
1             66          2146  ...     207362439        249924007
2              0             0  ...    3529373752       3202242038
3            232         10926  ...      57924025         29309323
4           1478         20434  ...     154108436         43507078
..           ...           ...  ...           ...              ...
95           604         34968  ...    4621149950       6831490734
96           523          9463  ...    2432212861       3765970601
97            22          1681  ...     175665367        194766424
98            49          1128  ...      48707071        184248580
99             0          3271  ...     183420766        320475018

[100 rows x 12 columns]


In [116]:
###연령-판매건수 리스트로 뽑기

data_sample2=data[['기준_년_코드','상권_구분_코드_명','서비스_업종_코드_명','연령대_10_매출_건수']]
data2_sorted_by_index = data_sample2.sort_values(by=['연령대_10_매출_건수'], ascending=False)
print(data2_sorted_by_index.head(100))

data_sample2=data[['기준_년_코드','상권_구분_코드_명','서비스_업종_코드_명','연령대_20_매출_건수']]
data2_sorted_by_index = data_sample2.sort_values(by=['연령대_20_매출_건수'], ascending=False)
print(data2_sorted_by_index.head(100))

data_sample2=data[['기준_년_코드','상권_구분_코드_명','서비스_업종_코드_명','연령대_30_매출_건수']]
data2_sorted_by_index = data_sample2.sort_values(by=['연령대_30_매출_건수'], ascending=False)
print(data2_sorted_by_index.head(100))

data_sample2=data[['기준_년_코드','상권_구분_코드_명','서비스_업종_코드_명','연령대_40_매출_건수']]
data2_sorted_by_index = data_sample2.sort_values(by=['연령대_40_매출_건수'], ascending=False)
print(data2_sorted_by_index.head(100))

data_sample2=data[['기준_년_코드','상권_구분_코드_명','서비스_업종_코드_명','연령대_50_매출_건수']]
data2_sorted_by_index = data_sample2.sort_values(by=['연령대_50_매출_건수'], ascending=False)
print(data2_sorted_by_index.head(100))

data_sample2=data[['기준_년_코드','상권_구분_코드_명','서비스_업종_코드_명','연령대_60_이상_매출_건수']]
data2_sorted_by_index = data_sample2.sort_values(by=['연령대_60_이상_매출_건수'], ascending=False)
print(data2_sorted_by_index.head(100))


########20대와 60대 이상 소비건수 제일 많아 (자기를 위한 소비)

#data01 = pd.DataFrame(sale_list)  
#sale_list.to_csv('/content/drive/My Drive/Colab Notebooks/data/revised_data/sale_list.csv') 

#년도-업종 리스트로 뽑기
#store_list= data1_sorted_by_index02[['기준_년_코드','상권_구분_코드_명','서비스_업종_코드_명']].head(100)
#print(store_list)

#data02 = pd.DataFrame(store_list) 
#store_list.to_csv('/content/drive/My Drive/Colab Notebooks/data/revised_data/store_list.csv') 


       기준_년_코드 상권_구분_코드_명 서비스_업종_코드_명  연령대_10_매출_건수
13559     2020       골목상권  컴퓨터및주변장치판매        140639
30803     2020       골목상권         핸드폰        133460
64061     2020       골목상권         핸드폰        133298
37865     2020       발달상권         PC방         89360
40090     2020       발달상권       커피-음료         86155
...        ...        ...         ...           ...
8185      2020       발달상권         PC방         21198
4515      2020       발달상권         PC방         21093
23956     2020       골목상권         PC방         21069
15905     2020       골목상권         편의점         20915
39961     2020       발달상권         편의점         20801

[100 rows x 4 columns]
       기준_년_코드 상권_구분_코드_명 서비스_업종_코드_명  연령대_20_매출_건수
40090     2020       발달상권       커피-음료       1184669
42870     2020       발달상권         편의점       1143049
8840      2020       발달상권        일반의류       1105057
9601      2020       발달상권         편의점       1090943
6839      2020       발달상권       커피-음료       1071003
...        ...        ...         ...   

In [120]:
###연령-판매금액 리스트로 뽑기
data_sample2=data[['기준_년_코드','상권_구분_코드_명','서비스_업종_코드_명','연령대_10_매출_금액']]                                           
data01_sorted_by_index = data_sample2.sort_values(by=['연령대_10_매출_금액'], ascending=False)
print(data2_sorted_by_index.head(100))

data_sample2=data[['기준_년_코드','상권_구분_코드_명','서비스_업종_코드_명','연령대_20_매출_금액']]
data02_sorted_by_index = data_sample2.sort_values(by=['연령대_20_매출_금액'], ascending=False)
print(data2_sorted_by_index.head(100))

data_sample2=data[['기준_년_코드','상권_구분_코드_명','서비스_업종_코드_명','연령대_30_매출_금액']]
data03_sorted_by_index = data_sample2.sort_values(by=['연령대_30_매출_금액'], ascending=False)
print(data2_sorted_by_index.head(100))

data_sample2=data[['기준_년_코드','상권_구분_코드_명','서비스_업종_코드_명','연령대_40_매출_금액']]
data04_sorted_by_index = data_sample2.sort_values(by=['연령대_40_매출_금액'], ascending=False)
print(data2_sorted_by_index.head(100))

data_sample2=data[['기준_년_코드','상권_구분_코드_명','서비스_업종_코드_명','연령대_50_매출_금액']]
data05_sorted_by_index = data_sample2.sort_values(by=['연령대_50_매출_금액'], ascending=False)
print(data2_sorted_by_index.head(100))

data_sample2=data[['기준_년_코드','상권_구분_코드_명','서비스_업종_코드_명','연령대_60_이상_매출_금액']]
data06_sorted_by_index = data_sample2.sort_values(by=['연령대_60_이상_매출_금액'], ascending=False)
print(data2_sorted_by_index.head(100))

       기준_년_코드 상권_구분_코드_명 서비스_업종_코드_명  연령대_60_이상_매출_건수
3606      2020       전통시장         청과상          1385663
8762      2020       발달상권       수산물판매          1276543
42026     2020       발달상권       수산물판매          1156656
3660      2020       전통시장         청과상          1089070
36861     2020       전통시장         청과상           766049
...        ...        ...         ...              ...
3662      2020       전통시장        육류판매            99536
6705      2020       발달상권         의약품            98052
191       2020       관광특구         화장품            97699
37578     2020       전통시장        반찬가게            97369
31245     2020       골목상권        일반의류            96507

[100 rows x 4 columns]
       기준_년_코드 상권_구분_코드_명 서비스_업종_코드_명  연령대_60_이상_매출_건수
3606      2020       전통시장         청과상          1385663
8762      2020       발달상권       수산물판매          1276543
42026     2020       발달상권       수산물판매          1156656
3660      2020       전통시장         청과상          1089070
36861     2020       전통시장         청과상    

In [ ]:

###연령리스트, 각 행 비율....'해당업종' 이용연령대 추출
#age_list=data2[['catm','age']]
#age_val=age_list.values.tolist()

#print(age_val)
#data2['age'].value_counts(normalize=True)

#data03 = pd.DataFrame(age_list) 
#age_list.to_csv('/content/drive/My Drive/Colab Notebooks/data/revised_data/age_list.csv') 

[['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['기호식품', '20'], ['기호식품', '20'], ['기호식품', '20'], ['기호식품', '20'], ['기호식품', '20'], ['기호식품', '20'], ['기호식품', '20'], ['기호식품', '20'], ['기호식품', '20'], ['기호식품', '20'], ['기호식품', '20'], [